In [1]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")


Libraries imported successfully......


### i. Player data Current Season


In [2]:
### Function to return player url 
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'
    
    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)
    
    return formatted_url

# Example usage:
player_id = 308  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


https://fantasy.premierleague.com/api/element-summary/308/


In [3]:
#This cell imports individual player information
r = requests.get(format_player_url(308))
player_json = r.json()
player_fixtures_df = pd.DataFrame(player_json['fixtures'])
player_history_df = pd.DataFrame(player_json['history'])
player_history_past_df = pd.DataFrame(player_json['history_past'])



In [7]:
# List of columns to drop
columns_to_drop = ['bonus', 'bps', 'influence', 'creativity', 'kickoff_time',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out']

# Use the drop method to remove the specified columns
player_history_df.drop(columns=columns_to_drop, inplace=True)


In [9]:
player_history_df


,element,fixture,opponent_team,total_points,was_home,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves
0,308,9,7,5,False,1,1,1,76,0,1,0,1,0,0,0,0,0,0
1,308,14,3,5,True,3,1,2,85,1,0,0,1,0,0,1,0,0,0
2,308,29,15,5,False,1,2,3,90,0,1,0,1,0,0,0,0,0,0
3,308,37,2,8,True,3,0,4,90,1,0,1,0,0,0,0,0,0,0
4,308,50,20,10,False,1,3,5,90,0,2,0,1,0,0,0,0,0,0
5,308,57,19,10,True,3,1,6,90,1,0,0,1,0,0,0,0,0,0
6,308,68,18,1,False,2,1,7,73,0,0,0,1,0,0,0,1,0,0
7,308,72,5,15,False,2,2,8,90,2,0,0,2,0,0,0,0,0,0
